In [1]:
import warnings
warnings.filterwarnings('ignore')

from TGA.utils import Dataset, GraphsizePretrained
#from tqdm import tqdm
from tqdm.notebook import tqdm
from time import time
from sklearn.preprocessing import OneHotEncoder
import numpy as np

from collections import Counter
import networkx as nx

In [2]:
%%time
graph_builder = GraphsizePretrained(w=2, verbose=True,stopwords='keep',
                   pretrained_vec='/home/mangaravite/Documentos/Universidade/LBD/pretrained_vectors/glove/glove.6B.300d.txt')

400000it [00:23, 17351.22it/s]


CPU times: user 22.6 s, sys: 727 ms, total: 23.4 s
Wall time: 23.3 s


In [184]:
dataset = Dataset('/home/mangaravite/Documentos/datasets/classification/datasets/imdb_reviews/')
fold = next(dataset.get_fold_instances(10, with_val=False))
fold._fields, len(fold.X_train)

(('X_train', 'y_train', 'X_test', 'y_test'), 313569)

In [185]:
def fit(self, X, y):
    self.N = len(X)
    y_train = self.le.fit( sorted(list(set(y))) ).transform(y)
    self.n_class = len(self.le.classes_)
    
    self.class_term_freqs = Counter()
    self.term_freqs = Counter()
    self.class_freqs = Counter(y_train)

    docs = list(map(self.analyzer.build_analyzer(), self.progress_bar(X)))
    self.node_mapper = dict()
    pairs = list( zip( docs, y_train ) )
    for (doc,y) in self.progress_bar(pairs):

        doc_in_terms = set(filter( lambda x: x in self.embeddings_dict, doc))
        terms_by_nid = list(map(lambda x: self.node_mapper.setdefault(x, len(self.node_mapper)), doc_in_terms))

        self.term_freqs.update( terms_by_nid )
        
        list_of_edges = list(map( lambda x: (y, x), terms_by_nid ))
        self.class_term_freqs.update( list_of_edges )

    return self

In [186]:
fit(graph_builder,fold.X_train, fold.y_train)

100%|██████████| 313569/313569 [01:06<00:00, 4747.81it/s]


GraphsizePretrained(encoding=None,
                    pretrained_vec='/home/mangaravite/Documentos/Universidade/LBD/pretrained_vectors/glove/glove.6B.300d.txt',
                    stopwords='keep', verbose=None)

In [187]:
from scipy.sparse import coo_matrix

In [188]:
triples= [ (x,y,v) for ((y,x), v) in graph_builder.class_term_freqs.items() ]
terms, clss, fresq = list(zip(*triples))
N_all  = np.sum(fresq)
n_terms= len(graph_builder.node_mapper)
n_class=graph_builder.n_class
cjct_terms_lbls = coo_matrix((fresq, (terms, clss)), shape=(n_terms, n_class))/N_all

In [189]:
labels_priors = cjct_terms_lbls.sum(axis=0)
terms_priors  = cjct_terms_lbls.sum(axis=1)

In [190]:
# Matrix of indepencende P(x)*P(y)
indp_mtrx = terms_priors.reshape(-1,1)*labels_priors.reshape(-1,1).T


In [191]:
terms_priors.shape, labels_priors.shape, cjct_terms_lbls.shape, indp_mtrx.shape

((82928, 1), (1, 10), (82928, 10), (82928, 10))

In [192]:
pmi = coo_matrix(cjct_terms_lbls/indp_mtrx)
pmi.data = np.log2(pmi.data)
pmi.A

array([[-0.82292229, -0.71277767, -0.5917258 , ...,  0.0288091 ,
         0.30158907,  0.59117272],
       [-0.98877499, -0.8403062 , -0.67573364, ...,  0.23836716,
         0.24228313,  0.10020134],
       [ 0.44066138,  0.41918953,  0.31333562, ..., -0.13999841,
        -0.17431109, -0.22273485],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         3.02815832,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [193]:
hxy = coo_matrix(cjct_terms_lbls)
hxy.data = -np.log2(hxy.data) # h(x,y)= -log2 P(x,y)

npmi= coo_matrix(pmi)
npmi.data = pmi.data/hxy.data #npmi(x,y) = pmi(x,y)/h(x,y)
npmi.min(), npmi.max(), npmi.A

(-0.26433051821771325,
 0.25828231590288275,
 array([[-0.05776433, -0.03964136, -0.03977587, ...,  0.00145702,
          0.02578555,  0.0340682 ],
        [-0.06223679, -0.06590377, -0.05239255, ...,  0.01695977,
          0.01569586,  0.00595598],
        [ 0.03906779,  0.0249523 ,  0.01515927, ..., -0.01126963,
         -0.01545525, -0.0187372 ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.11862313,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]))

# Exemplos de termos disciminantes

In [194]:
sum_abs = np.abs(npmi.A).sum(axis=1)
sorted_index = (-sum_abs).argsort()
sorted_index

array([44529, 46731, 34765, ...,   468,    37,    50])

In [195]:
inverted_mapper = { v:k for (k,v) in graph_builder.node_mapper.items() }

In [196]:
sum_abs.argmax(), inverted_mapper[sum_abs.argmax()], npmi.A[sum_abs.argmax()]

(44529,
 'hobgoblins',
 array([ 0.21451797,  0.1323334 ,  0.02350575, -0.09971513, -0.13465781,
        -0.16801612, -0.16669893, -0.13003633,  0.        , -0.07867374]))

In [197]:
n=100
list(zip([inverted_mapper[x] for x in sorted_index[:n]], np.round(npmi.A[list(sorted_index[:n])], decimals=1).tolist()))

[('hobgoblins', [0.2, 0.1, 0.0, -0.1, -0.1, -0.2, -0.2, -0.1, 0.0, -0.1]),
 ('mst', [0.1, 0.1, 0.1, 0.0, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1]),
 ('bloodrayne', [0.2, 0.1, 0.0, 0.0, 0.0, -0.0, -0.0, -0.1, -0.2, -0.1]),
 ('insultingly', [0.1, 0.1, 0.2, 0.1, 0.0, -0.0, -0.1, -0.1, -0.1, -0.2]),
 ('hayao', [-0.2, -0.1, -0.2, -0.2, -0.1, -0.1, -0.0, 0.0, 0.1, 0.1]),
 ('stupidest', [0.1, 0.1, 0.1, 0.0, -0.0, -0.0, -0.1, -0.1, -0.2, -0.1]),
 ('mavens', [0.0, 0.0, -0.1, -0.1, -0.1, -0.1, -0.1, -0.2, -0.1, 0.1]),
 ('stupider', [0.1, 0.1, 0.0, 0.1, 0.0, -0.1, -0.1, -0.1, -0.1, -0.2]),
 ('turd', [0.1, 0.1, 0.1, 0.0, -0.0, -0.1, -0.1, -0.1, -0.1, -0.1]),
 ('zatoichi', [-0.1, -0.1, -0.1, -0.1, -0.1, -0.1, 0.0, 0.1, 0.0, -0.1]),
 ('unengaging', [0.1, 0.1, 0.1, 0.1, 0.0, 0.0, -0.1, -0.1, -0.2, 0.0]),
 ('masochists', [0.2, 0.0, 0.1, -0.1, 0.0, -0.1, -0.1, 0.0, -0.1, -0.1]),
 ('moreland', [-0.0, 0.1, 0.1, 0.1, 0.1, 0.0, 0.0, -0.1, -0.2, -0.1]),
 ('godawful', [0.2, 0.1, 0.1, 0.0, -0.0, -0.0, -0.1, -0.1, 

# Infering semantic-syntatic

Infer relationship between nPMI and Label->Term 

In [198]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

device=torch.device('cuda')

## Important functions

In [199]:
def masked_softmax(vec, mask=None, dim=1, epsilon=1e-5):
    exps = torch.exp(vec)
    if mask is None:
        mask = vec > 0.
    masked_exps = exps * mask.float()
    masked_sums = masked_exps.sum(dim, keepdim=True) + epsilon
    return (masked_exps/masked_sums)

## Model

In [200]:
class GlobalPMI(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes, drop=0.3, drop_global=.5, loss=None, device=torch.device('cuda')):
        super(GlobalPMI, self).__init__()
        
        self.hidden_dim = hidden_dim
        
        self.global_term_encoder = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d( hidden_dim ),
            nn.Dropout(drop_global),
            nn.Linear(hidden_dim, hidden_dim)
        ).to(device)
        
        self.local_term_encoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(hidden_dim, hidden_dim)
        ).to(device)
        
        self.label_encoder = nn.Sequential(
            nn.Linear(n_classes, hidden_dim),
            nn.BatchNorm1d( hidden_dim ),
            nn.Dropout(drop),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d( hidden_dim )
        ).to(device)
        
        self.y = torch.eye( n_classes, n_classes ).to(device)
        
        self.att     = nn.Linear(hidden_dim, hidden_dim).to(device)
        """self.att_act = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Dropout(drop),
            nn.Linear(hidden_dim, 1)
        )"""
        
        self.doc_encoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Dropout(drop),
            nn.Linear(hidden_dim, hidden_dim)
        ).to(device)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Dropout(drop),
            nn.Linear(hidden_dim, n_classes),
            nn.Softmax()
        ).to(device)
        
        self.loss = loss.to(device)
        self.device = device
        
    def forward(self, term_emb, term_mask, doc_mask):
        batch_size = doc_mask.max()+1
        
        h_label = self.label_encoder(self.y)
        h_term  = self.global_term_encoder(term_emb)
        
        h_local_term = self.local_term_encoder(h_term[term_mask])
        docs = torch.rand((batch_size, self.hidden_dim))
        
        for docid in range(batch_size):
            doc_term_emb = h_local_term[doc_mask == docid]
            alpha = self.att(doc_term_emb)
            alpha = F.softmax(doc_term_emb)
            
            #act = self.att_act(alpha)
            #act = masked_softmax( act )
            #docs[docid] = (act*alpha*doc_term_emb).mean(dim=0)
            
            docs[docid] = (alpha*doc_term_emb).mean(dim=0)
            
        docs = self.doc_encoder( docs.to(self.device) )
        docs_probs = self.fc( docs )
        return h_term, h_label, docs, docs_probs

In [201]:
from TGA.lossweight import cross_entropy
class NpairLoss(nn.Module):
    """the multi-class n-pair loss"""
    def __init__(self, l2_reg=0.02):
        super(NpairLoss, self).__init__()
        self.l2_reg = l2_reg

    def forward(self, anchor, target, positive=None):
        batch_size = anchor.size(0)
        target = target.view(target.size(0), 1)

        target = (target == torch.transpose(target, 0, 1)).float()
        target = target / torch.sum(target, dim=1, keepdim=True).float()

        if positive is not None:
            logit = torch.matmul(anchor, torch.transpose(positive, 0, 1))
            l2_loss = torch.sum(anchor**2) / batch_size + torch.sum(positive**2) / batch_size
        else:
            logit = torch.matmul(anchor, torch.transpose(anchor, 0, 1))
            l2_loss = torch.sum(anchor**2) / batch_size
        
        loss_ce = cross_entropy(logit, target)

        loss = loss_ce + self.l2_reg*l2_loss*0.25
        return loss
class TGALoss(torch.nn.Module):
    def __init__(self, l2_reg=5e-4):
        super(TGALoss, self).__init__()
        self.sigma    = nn.Parameter(torch.ones(4))
        self.mse_loss = nn.MSELoss()
        self.ce_loss  = nn.CrossEntropyLoss()
        self.npl_loss = NpairLoss(l2_reg=l2_reg)
        self.dst_loss = SelfDistLoss()

    def forward(self, loss_term_npmi, loss_doc_class, loss_cross_ent, loss_self_dist, explain=False):
        
        loss_term_npmi = loss_term_npmi/(self.sigma[0]**2) + torch.log(self.sigma[0])
        loss_doc_class = loss_doc_class/(2.*self.sigma[1]**2) + torch.log(self.sigma[1])
        loss_cross_ent = loss_cross_ent/(2.*self.sigma[2]**2) + torch.log(self.sigma[2])
        loss_self_dist = loss_self_dist/(self.sigma[3]**2) + torch.log(self.sigma[3])
        
        loss = (loss_term_npmi + loss_doc_class + loss_cross_ent + loss_self_dist).mean()
        
        if explain:
            explanation = {
                'term_npmi': (loss_term_npmi.mean().item(), self.sigma[0].item()),
                'doc_class': (loss_doc_class.mean().item(), self.sigma[1].item()),
                'cross_ent': (loss_cross_ent.mean().item(), self.sigma[2].item()),
                'self_dist': (loss_self_dist.mean().item(), self.sigma[3].item()),
            }
            return loss, explanation
        return loss
class SelfDistLoss(nn.Module):
    def __init__(self, l2_reg=0.02, eps = 0.00003):
        super(SelfDistLoss, self).__init__()
        self.l2_reg = l2_reg
        self.eps = eps
        
    def forward(self, hiddens):
        L = torch.matmul(hiddens.T, hiddens)
        L = F.sigmoid(L)
        L = (L - L.diag()).float()
        L = F.relu(L)
        L = ( L > 0. ).float() * torch.exp( L )
        #L = F.normalize(L)
        
        values = L.sum(axis=1)
        svalue = max((values > 0.).sum(), self.eps)

        return values.sum()/svalue # AVG of non-zero values

In [202]:
tgaloss = TGALoss()

globalpmi = GlobalPMI(300, 300, dataset.nclass, drop=0.1, loss=tgaloss)
globalpmi = globalpmi.cuda()
globalpmi.cuda()

GlobalPMI(
  (global_term_encoder): Sequential(
    (0): Linear(in_features=300, out_features=300, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=300, out_features=300, bias=True)
  )
  (local_term_encoder): Sequential(
    (0): Linear(in_features=300, out_features=300, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=300, out_features=300, bias=True)
  )
  (label_encoder): Sequential(
    (0): Linear(in_features=10, out_features=300, bias=True)
    (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=300, out_features=300, bias=True)
    (4): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (att): Linear(in_features=300, out_features=300, bias=True)
  (doc_encoder): Sequ

In [203]:
def transform(self, X):
    analyzer = self.analyzer.build_analyzer()
    local_mapper = dict()
    term_emb = []
    term_mask = []
    doc_mask = []
    for i,doc in enumerate(X):
        doc_in_terms = analyzer(doc)
        doc_in_terms = set(filter( lambda x: x in self.embeddings_dict, doc_in_terms))
        #print(doc_in_terms)
        for tid in doc_in_terms:
            if tid not in local_mapper:
                local_mapper[tid] = len(local_mapper)
                term_emb.append( self.embeddings_dict[tid] )
            term_mask.append( local_mapper[tid] )
            doc_mask.append( i )
    return term_emb, term_mask, doc_mask, local_mapper

In [204]:
def collate(param):
    X, y = zip(*param)
    term_emb, term_mask, doc_mask, local_mapper = transform(graph_builder, X)
    
    return term_emb, term_mask, doc_mask, local_mapper, y

def get_term_emb( local_mapper, term_emb ):
    local_terms = list(zip(*sorted(local_mapper.items(), key=lambda x: x[1] )))[0]

    idx_local  = [ local_mapper[x] for x in local_terms if x in graph_builder.node_mapper ] 
    idx_global = [ graph_builder.node_mapper[x] for x in local_terms if x in graph_builder.node_mapper ]

    filtered_npmi = npmi.A[ idx_global ]

    sum_abs = np.abs(filtered_npmi).sum(axis=1)
    sorted_index = (-sum_abs).argsort()
    sorted_index = sorted_index[ :int(p*len(idx_global)) ]

    term_emb_pred  = term_emb[[idx_local[x] for x in sorted_index]]
    term_npmi      = torch.Tensor(filtered_npmi[sorted_index])
    
    return term_emb_pred, term_npmi


In [205]:
batch_size=64
p=0.3
margin=0.5

In [206]:
#optimizer = optim.SGD( globalpmi.parameters(), lr=5e-3, momentum=0.9)
optimizer = optim.AdamW( globalpmi.parameters(), lr=5e-3, weight_decay=5e-3)

mse_loss = nn.MSELoss().to(device)
ce_loss  = nn.CrossEntropyLoss().to(device)
npl_loss = NpairLoss(l2_reg=5e-4).to(device)

In [207]:
best = None
nepochs = 10
globalpmi.train()
for e in tqdm(range(nepochs), total=nepochs):
    epoch_loss = 0
    epoch_loss1 = 0
    epoch_loss2 = 0
    epoch_loss3 = 0
    epoch_loss4 = 0
    TP = 0
    Ndocs = 0
    y_train = graph_builder.le.transform(fold.y_train)
    data_loader = DataLoader(list(zip(fold.X_train, y_train)), batch_size=batch_size,
                             shuffle=True, collate_fn=collate, num_workers=4)
    with tqdm(total=len(fold.y_train), smoothing=0.) as pbar:
        total = 1
        correct = 0
        for i, (term_emb, term_mask, doc_mask, local_mapper, y) in enumerate(data_loader):
            term_emb = torch.Tensor(term_emb).to(device)
            term_mask = torch.LongTensor(term_mask).to(device)
            doc_mask = torch.LongTensor(doc_mask).to(device)
            y = torch.LongTensor(y).to(device)
            
            h_term, h_label, h_docs, y_probs = globalpmi(term_emb, term_mask, doc_mask)
            
            h_term, h_label, h_docs, y_probs = h_term.to(device), h_label.to(device), h_docs.to(device), y_probs.to(device)
            
            term_emb_pred, term_npmi = get_term_emb( local_mapper, term_emb )
            term_emb_pred, term_npmi = term_emb_pred.to(device), term_npmi.to(device)
            
            loss1 = tgaloss.mse_loss( term_npmi, torch.matmul( term_emb_pred, h_label.T ) )
            loss2 = tgaloss.npl_loss(h_docs, y, h_label[y])
            loss3 = tgaloss.ce_loss( y_probs, y )
            loss4 = tgaloss.dst_loss( h_label )
            
            epoch_loss1 += loss1
            epoch_loss2 += loss2
            epoch_loss3 += loss3
            epoch_loss4 += loss4
            
            if loss1 < margin:
                loss1 = margin
            
            #loss = loss2 + loss3
            loss = tgaloss( loss1, loss2, loss3, loss4 )
            
            epoch_loss += loss
            
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            
            TP += (y_probs.argmax(dim=1)==y).sum()
            Ndocs += len(y)
            
            pbar.update( len(y) )
            pbar.set_description_str(f'Lss: {epoch_loss/(i+1):.4} \
                    Lss_i: ({epoch_loss1/(i+1):.4};{epoch_loss2/(i+1):.4};{epoch_loss3/(i+1):.4};{epoch_loss4/(i+1):.4})')
            print(f'Acc: {TP/Ndocs:.4}, Lss1: {loss1:.4}, Lss2: {loss2:.4}, Lss3: {loss3:.4}, Lss4: {loss4:.4}' , end='\r')

Acc: 0.2769, Lss1: 0.5, Lss2: 3.918, Lss3: 2.222, Lss4: 1.002033



RuntimeError: CUDA out of memory. Tried to allocate 14.00 MiB (GPU 0; 3.95 GiB total capacity; 3.12 GiB already allocated; 7.06 MiB free; 3.31 GiB reserved in total by PyTorch)

In [ ]:
#ACM: Acc: 0.5857, Lss1: 0.01755, Lss2: 0.004775
self_label = F.sigmoid(torch.matmul(h_label.T, h_label))
F.relu(self_label-self_label.diag()).sum()

In [ ]:
self_dist = SelfDistLoss()
self_dist( h_label )

In [138]:
loss1

tensor(0.5052, device='cuda:0', grad_fn=<MseLossBackward>)

In [ ]:
h_label.shape, y, graph_builder.n_class

In [ ]:
doc_terms_emb = term_emb[ term_mask[doc_mask == 0] ]

In [ ]:
doc_terms_emb.sum(dim=0).shape

In [ ]:
F.softmax(doc_terms_emb.T).T

In [ ]:
y_probs.argmax(dim=1)